In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import json
import pickle as pkl
import numpy as np
import torch_geometric as pyg
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_dense_adj
import torch
import os
import re

os.chdir("..")

from scripts.utils import *


# Process **One** Movie

In [2]:
tr_len = 1.3 # duration of fMRI tr
n_nodes = 414 # number of nodes in the graph
movie = "AfterTheRain" # Name of the movie to process
delay_tr = 4 # Delay in TR after which the emotion will be detectable ofn the fMRI data
node_feat_method = "singlefmri" #method to use for node features
intial_adj_method = "clique" #methos to use for graph connectivity intializatrion
base_save_dir = "/home/dalai/GNN_E/data/processed"

# Load necessary data

In [3]:
data = pd.read_csv("/media/RCPNAS/MIP/Michael/ml-students2024/data/fmri414_compiled/AfterTheRain_compiled414.csv")
data.head(2)

,filename,vindex,full_region,partial_region,score,movie_name,parcel,id,session
0,TC_414_sub-S25_AfterTheRain.csv,0,7Networks_LH_Vis_1,Vis,-28.029734,AfterTheRain,400,sub-S25,ses-1
1,TC_414_sub-S25_AfterTheRain.csv,0,7Networks_LH_Vis_1,Vis,-28.084551,AfterTheRain,400,sub-S25,ses-1


In [4]:
labels = pd.read_csv("/media/RCPNAS/MIP/Michael/ml-students2024/data/emo_labels/Annot_13_AfterTheRain_stim.tsv", sep="\t", header=None)
labels.head(2)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.142779,0.281039,0.250808,0.330057,-0.341303,-0.076385,1.608596,-0.027852,-0.552002,-0.046603,...,-0.092547,0.380354,-0.154463,-0.041718,-0.652966,0.108885,0.336159,-0.336729,0.276114,-0.135235
1,0.015013,0.494496,0.249501,0.452919,-0.385233,0.149844,1.759461,-0.145757,-0.384084,-0.902979,...,-0.027584,0.501055,-0.414479,-0.165635,-1.018303,0.296978,0.325535,-0.252170,0.333992,-0.578877


In [5]:
with open("/media/RCPNAS/MIP/Michael/ml-students2024/data/run_onsets.pkl", "rb") as file:
    delta_time = pkl.load(file)

# Delta time of the current movie
delta_time_single = delta_time[movie]
delta_time_single #(start of the movie in sec, duration of the movie in sec)

{'S27': (76, 382),
 'S19': (75, 382),
 'S06': (75, 382),
 'S02': (75, 382),
 'S23': (75, 382),
 'S09': (75, 382),
 'S16': (75, 382),
 'S28': (75, 382),
 'S32': (75, 382),
 'S13': (75, 382),
 'S07': (75, 382),
 'S26': (75, 382),
 'S22': (75, 382),
 'S03': (76, 382),
 'S17': (75, 382),
 'S29': (75, 382),
 'S08': (75, 382),
 'S04': (75, 382),
 'S25': (75, 382),
 'S31': (75, 382),
 'S10': (75, 382),
 'S14': (75, 382),
 'S21': (75, 382),
 'S24': (75, 382),
 'S05': (75, 382),
 'S11': (75, 382),
 'S30': (75, 382),
 'S15': (75, 382),
 'S01': (75, 382),
 'S20': (75, 382)}

# Add Classification Labels

In [6]:
resolved_labels = labels.copy()

# MEHTOD 1: TAKE THE HIGHEEST SCORE
resolved_labels["single_label"] = labels.idxmax(axis=1)

# MEHTOD 2: TAKE THE HIGHER THAN THR
    # attnetion we can add "neutrality" emotion
    # chc ig neurtrlaity is alredy insiude the 50 emo
thr_emo = 1
resolved_labels["multi_label"] = labels.apply(
    lambda row: [1 if value > thr_emo else 0 for value in row], axis=1
)

resolved_labels.head(6)

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,single_label,multi_label
0,0.142779,0.281039,0.250808,0.330057,-0.341303,-0.076385,1.608596,-0.027852,-0.552002,-0.046603,...,-0.154463,-0.041718,-0.652966,0.108885,0.336159,-0.336729,0.276114,-0.135235,6,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,0.015013,0.494496,0.249501,0.452919,-0.385233,0.149844,1.759461,-0.145757,-0.384084,-0.902979,...,-0.414479,-0.165635,-1.018303,0.296978,0.325535,-0.252170,0.333992,-0.578877,6,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,0.111895,0.379248,0.301687,0.408731,-0.176680,0.071492,1.389783,-0.254050,-0.434333,-1.026354,...,-0.759266,-0.114860,-1.350485,0.218283,0.331140,-0.338974,0.302379,-0.411378,6,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,0.003497,0.436360,0.348070,0.260503,0.043851,0.112154,1.395174,-0.403072,-0.411058,-1.142819,...,-0.905446,-0.147157,-1.813156,0.240899,0.326921,-0.250029,-0.498317,-0.506159,6,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,0.047341,0.398398,0.408016,0.029337,0.051369,0.089824,-0.004560,-0.435806,-0.423261,-1.183318,...,-0.805941,-0.123036,-1.928912,0.117960,0.330476,-0.341319,-0.926475,-0.436339,34,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,0.055064,0.418973,0.447900,0.023118,-0.075438,0.099314,-0.539354,-0.417008,-0.416852,-1.235597,...,-0.811525,-0.142840,-1.928831,0.078924,0.327308,-0.247379,-0.877334,-0.337143,34,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


# Add timestap in frmi data, both in "tr" and "sec"

In [7]:
# Add TR columns for **each region**
data["timestamp_tr"] = data.groupby(["id", "vindex"]).cumcount()
data["timestamp_sec"] = data["timestamp_tr"] * tr_len

data

,filename,vindex,full_region,partial_region,score,movie_name,parcel,id,session,timestamp_tr,timestamp_sec
0,TC_414_sub-S25_AfterTheRain.csv,0,7Networks_LH_Vis_1,Vis,-28.029734,AfterTheRain,400,sub-S25,ses-1,0,0.0
1,TC_414_sub-S25_AfterTheRain.csv,0,7Networks_LH_Vis_1,Vis,-28.084551,AfterTheRain,400,sub-S25,ses-1,1,1.3
2,TC_414_sub-S25_AfterTheRain.csv,0,7Networks_LH_Vis_1,Vis,-1.884319,AfterTheRain,400,sub-S25,ses-1,2,2.6
3,TC_414_sub-S25_AfterTheRain.csv,0,7Networks_LH_Vis_1,Vis,-0.738570,AfterTheRain,400,sub-S25,ses-1,3,3.9
4,TC_414_sub-S25_AfterTheRain.csv,0,7Networks_LH_Vis_1,Vis,48.665913,AfterTheRain,400,sub-S25,ses-1,4,5.2
...,...,...,...,...,...,...,...,...,...,...,...
6691063,TC_414_sub-S28_AfterTheRain.csv,413,Right Accumbens,Sub,16.858381,AfterTheRain,14,sub-S28,ses-3,531,690.3
6691064,TC_414_sub-S28_AfterTheRain.csv,413,Right Accumbens,Sub,22.814312,AfterTheRain,14,sub-S28,ses-3,532,691.6
6691065,TC_414_sub-S28_AfterTheRain.csv,413,Right Accumbens,Sub,-12.612755,AfterTheRain,14,sub-S28,ses-3,533,692.9
6691066,TC_414_sub-S28_AfterTheRain.csv,413,Right Accumbens,Sub,31.981007,AfterTheRain,14,sub-S28,ses-3,534,694.2


# Graph
- of single movie
- of single subject
- of single timepoint

In [8]:
# All subjects ids
sub_ids = data["id"].sort_values().unique()
# Extract numbers using regex
pattern = r'sub-S(\d+)'
sub_ids = [re.search(pattern, id).group(1) for id in sub_ids if re.search(pattern, id)]
print(sub_ids)

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '13', '14', '15', '16', '17', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32']


In [9]:
#graphs = {}
save_dir = os.path.join(base_save_dir,"all_graphs", f"{intial_adj_method}_{node_feat_method}")
os.makedirs(save_dir, exist_ok=True)  # Create intermediate directories if they don't exist


for sub in sub_ids:

    ################################
    # Extarct subejct data

    print(f"Subject{sub}, movie: {movie}")

    # Ectract start (in sec) of the movie and the duation of the movie in sec
    delta_time_sub = delta_time_single[f"S{sub}"]
    print(delta_time_sub)

    start_movie_sec = delta_time_sub[0]
    lenght_movie_sec = delta_time_sub[1]
    end_movie_sec = start_movie_sec + lenght_movie_sec
    print(f"Movie shown at {start_movie_sec} sec. Duration of the movie {lenght_movie_sec} sec.")

    # Ectartc scores of one subject
    data_single_sub = data[data["id"] == f"sub-S{sub}"]
    print(f"Shape data single subject: {data_single_sub.shape}")

    # labels single user and single movie
    label_single_sub = resolved_labels["single_label"].values
    print(f"Shape labels single subject: {label_single_sub.shape} {label_single_sub[:5]}(one for each second of the duration of the movie)")

    ##############################

    # Align Labels

    # time (in sec) after which the movie is shown to the subj, PLUS the delay (in sec) after which the signal is vivible in frmi 
    # Thus, time (in sec) where the first label should be put
    start_movie_sec_shifted = start_movie_sec + tr_len * delay_tr
    print(f"Movie started at {start_movie_sec} sec, but fMRI effect will be visible at {start_movie_sec_shifted}")

    # Same for the end of the movie
    end_movie_sec_shifted = end_movie_sec + tr_len * delay_tr

    # Timestaps (in sec) from the shifted start of the movie, till the end of the movie
    # i.e. the timestap that we want to label
    timestap_to_label = data_single_sub[data_single_sub["timestamp_sec"] >= start_movie_sec_shifted]["timestamp_sec"].unique()
    timestap_to_label = timestap_to_label[timestap_to_label <= end_movie_sec_shifted][:-1] # ATTENTION: [-1] to not overcome the borders
    print(f"This are the timestamp to label (each one is a TR): {timestap_to_label[:4]}... Total {len(timestap_to_label)}")

    # Method choosen for aligning: NEAREST !!! (no interpolation done)
        # As the labels are index from 0, we have to shift the timestap 
    timestap_to_label_as_index =  timestap_to_label - np.floor(timestap_to_label[0]) # to be start form 0
    timestap_to_label_as_index = np.round(timestap_to_label_as_index)
    print(f"Index to use to select the labels: {timestap_to_label_as_index[:5]}... Total {len(timestap_to_label_as_index)}")

    # Select the label correpcifno to each timepoint
    labels_final_sub = label_single_sub[timestap_to_label_as_index.astype(int)]  # Convert to integer for indexing
    print(f"Final label per timepont (one for ech TR insde the movie duration): {labels_final_sub[:5]}... Total {len(labels_final_sub)}")

    # Create a df with the TR timestamp in sec and the raltive label (easier for later to label entier dataset)
    df_label_sub = pd.DataFrame({"timestamp_sec": timestap_to_label, "label": labels_final_sub, "index_used": timestap_to_label_as_index})
    display(df_label_sub.head(4))

    ######################
    
    #Filter df data to only have inside the time when the movie is shown
    data_single_sub = data_single_sub[(data_single_sub["timestamp_sec"] >= start_movie_sec_shifted) & (data_single_sub["timestamp_sec"] <= end_movie_sec_shifted)]

    # Label the datset with the labels
    data_single_sub_labels = pd.merge(df_label_sub, data_single_sub, on="timestamp_sec")

    display(data_single_sub_labels)

    #######################

    timepoints_tr = data_single_sub_labels["timestamp_tr"].unique()

    # Create one graph for each timestap
    for timepoint in timepoints_tr:

        # Only one timepoitn
        data_single_sub_labels_timepoint = data_single_sub_labels[data_single_sub_labels["timestamp_tr"] == timepoint]

        display(f"Shape DF of single sub of single timepoint : {data_single_sub_labels_timepoint.shape}") ##ROIs x cols

        # DECIDE NODE FEATURES
        if node_feat_method == "singlefmri":
            x = data_single_sub_labels_timepoint[["vindex", "score"]]
            #display(x)
            x_matrix = np.array(x["score"])
            x_matrix = torch.tensor(x_matrix, dtype=torch.float)

        # DDECIDE INITIAL CONNECTIVITY
        if intial_adj_method == "clique":
            adj_matrix_initial = np.ones((n_nodes, n_nodes)) - np.eye(n_nodes)
            adj_tensor = torch.tensor(adj_matrix_initial, dtype=torch.float)
            print(f"Shape adj matrix: {adj_tensor.shape}")

        # EXTARCT GRAPH LABEL
        single_graph_label = data_single_sub_labels_timepoint["label"].unique()[0]
        single_graph_label = torch.tensor(single_graph_label, dtype=torch.long)
        print(f"Label fo the current grpah: {single_graph_label}")

        # Create custum graph

        graph = GraphEmo(
                    x=x_matrix, 
                    edge_index=None, 
                    edge_attr=None, 
                    y=single_graph_label, 
                    adj = adj_tensor, 
                    movie=movie, 
                    subject=sub, 
                    timestamp_tr=timepoint
                )

        print(graph)

        # Save separate  graph

        file_path = os.path.join(save_dir, 
                                f"{movie}_{sub}_{timepoint  }.pt")# Construct the filename with extension
        torch.save(graph, file_path, pickle_protocol=4)# Save the graph
        print(f"Graph saved at: {file_path}")

        # put graph together
        #graphs.update({f"{movie}_{sub}_{timepoint:.1f}": graph})

        print(f"\n\n{sub} - {timepoint}")

        #break #fir ech timepoint

    break #for each sub

#torch.save(graphs, os.path.join(save_dir, f"{movie}.pth")) # to save all graphas of the same movie together


Subject01, movie: AfterTheRain
(75, 382)
Movie shown at 75 sec. Duration of the movie 382 sec.
Shape data single subject: (221076, 11)
Shape labels single subject: (382,) [ 6  6  6  6 34](one for each second of the duration of the movie)
Movie started at 75 sec, but fMRI effect will be visible at 80.2
This are the timestamp to label (each one is a TR): [80.6 81.9 83.2 84.5]... Total 293
Index to use to select the labels: [1. 2. 3. 4. 6.]... Total 293
Final label per timepont (one for ech TR insde the movie duration): [ 6  6  6 34 34]... Total 293


,timestamp_sec,label,index_used
0,80.6,6,1.0
1,81.9,6,2.0
2,83.2,6,3.0
3,84.5,34,4.0


,timestamp_sec,label,index_used,filename,vindex,full_region,partial_region,score,movie_name,parcel,id,session,timestamp_tr
0,80.6,6,1.0,TC_414_sub-S01_AfterTheRain.csv,0,7Networks_LH_Vis_1,Vis,-46.140919,AfterTheRain,400,sub-S01,ses-1,62
1,80.6,6,1.0,TC_414_sub-S01_AfterTheRain.csv,1,7Networks_LH_Vis_2,Vis,-3.570048,AfterTheRain,400,sub-S01,ses-1,62
2,80.6,6,1.0,TC_414_sub-S01_AfterTheRain.csv,2,7Networks_LH_Vis_3,Vis,-13.413118,AfterTheRain,400,sub-S01,ses-1,62
3,80.6,6,1.0,TC_414_sub-S01_AfterTheRain.csv,3,7Networks_LH_Vis_4,Vis,-18.172947,AfterTheRain,400,sub-S01,ses-1,62
4,80.6,6,1.0,TC_414_sub-S01_AfterTheRain.csv,4,7Networks_LH_Vis_5,Vis,-11.418571,AfterTheRain,400,sub-S01,ses-1,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...
121297,460.2,49,380.0,TC_414_sub-S01_AfterTheRain.csv,409,Right Putamen,Sub,23.405094,AfterTheRain,14,sub-S01,ses-2,354
121298,460.2,49,380.0,TC_414_sub-S01_AfterTheRain.csv,410,Right Pallidum,Sub,24.006180,AfterTheRain,14,sub-S01,ses-2,354
121299,460.2,49,380.0,TC_414_sub-S01_AfterTheRain.csv,411,Right Hippocampus,Sub,27.298269,AfterTheRain,14,sub-S01,ses-2,354
121300,460.2,49,380.0,TC_414_sub-S01_AfterTheRain.csv,412,Right Amygdala,Sub,36.601147,AfterTheRain,14,sub-S01,ses-2,354


'Shape DF of single sub of single timepoint : (414, 13)'

Shape adj matrix: torch.Size([414, 414])
Label fo the current grpah: 6
tensor([[  0,   0,   0,  ..., 413, 413, 413],
        [  1,   2,   3,  ..., 410, 411, 412]])
torch.Size([2, 170982])
tensor([1., 1., 1.,  ..., 1., 1., 1.])
torch.Size([170982])
GraphEmo(x=[414], edge_index=[2, 170982], edge_attr=[170982], y=6, movie='AfterTheRain', subject='01', timestamp_tr=62, adj=[414, 414])
Graph saved at: /home/dalai/GNN_E/data/processed/all_graphs/clique_singlefmri/AfterTheRain_01_62.pt


01 - 62


...

Graph saved at: /home/dalai/GNN_E/data/processed/all_graphs/clique_singlefmri/AfterTheRain_01_353.pt


01 - 353


'Shape DF of single sub of single timepoint : (414, 13)'

Shape adj matrix: torch.Size([414, 414])
Label fo the current grpah: 49
tensor([[  0,   0,   0,  ..., 413, 413, 413],
        [  1,   2,   3,  ..., 410, 411, 412]])
torch.Size([2, 170982])
tensor([1., 1., 1.,  ..., 1., 1., 1.])
torch.Size([170982])
GraphEmo(x=[414], edge_index=[2, 170982], edge_attr=[170982], y=49, movie='AfterTheRain', subject='01', timestamp_tr=354, adj=[414, 414])
Graph saved at: /home/dalai/GNN_E/data/processed/all_graphs/clique_singlefmri/AfterTheRain_01_354.pt

/home/dalai/GNN_E/scripts/utils.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(adj.nonzero().t().contiguous(), dtype=torch.long)  # Shape: [2, num_edges]
/home/dalai/GNN_E/scripts/utils.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_attr = torch.tensor(adj[edge_index[0, :], edge_index[1, :]], dtype=torch.float)





01 - 354


# Create Dataset

In [14]:
dataset = DatasetEmo("/home/dalai/GNN_E/data/processed/all_graphs/clique_singlefmri")
print(dataset)
print(len(dataset))

# Explore individual graph components
if len(dataset) > 0:
    for i in range(len(dataset)):  # Examine up to the first 2 graphs
        graph, label = dataset[i]
        #print(f"\nGraph {i} details:")
        
        # Check and print node features
        if graph.x is None:
            print("Node features (x) is None.")


        # Check and print edge indices
        if graph.edge_index is None or graph.edge_index.numel() == 0:
            print("Edge index is None or empty.")
 

        # Check and print edge attributes
        if graph.edge_attr is None or graph.edge_attr.numel() == 0:
            print("Edge attributes (edge_attr) is None or empty.")


        # Check and print label
        if label is None:
            print("Label (y) is None.")


        # Check and print additional attributes
        if graph.movie is None:
            print("Movie attribute is None.")

        
        if graph.subject is None:
            print("Subject attribute is None.")

        
        if graph.timestamp_tr is None:
            print("Timestamp attribute is None.")




DatasetEmo(293)
293


In [15]:
print(dataset[0])
print(type(dataset[0]))

#__get__ returns a tuple of (graph, label)
print(type(dataset[0][0]))
print(type(dataset[0][1]))

(GraphEmo(x=[414], edge_index=[2, 170982], edge_attr=[170982], y=34, movie='AfterTheRain', subject='01', timestamp_tr=296, adj=[414, 414]), tensor(34))
<class 'tuple'>
<class 'scripts.utils.GraphEmo'>
<class 'torch.Tensor'>


# Create Dataloader

In [16]:
batch_size = 32

In [19]:
dataloader = DataLoaderEmo(dataset= dataset, batch_size=batch_size, shuffle=True)

# Retrieve and print a single batch
for graphs, labels in dataloader:
    print(f'Batch size: {len(graphs)}')  # Number of graphs in the batch
    print(f'Graph object: {graphs}')  # Show the graph object
    print(f'Labels: {labels}')  # Show the labels

Batch size: 32
Graph object: (GraphEmo(x=[414], edge_index=[2, 170982], edge_attr=[170982], y=34, movie='AfterTheRain', subject='01', timestamp_tr=292, adj=[414, 414]), GraphEmo(x=[414], edge_index=[2, 170982], edge_attr=[170982], y=34, movie='AfterTheRain', subject='01', timestamp_tr=177, adj=[414, 414]), GraphEmo(x=[414], edge_index=[2, 170982], edge_attr=[170982], y=34, movie='AfterTheRain', subject='01', timestamp_tr=136, adj=[414, 414]), GraphEmo(x=[414], edge_index=[2, 170982], edge_attr=[170982], y=34, movie='AfterTheRain', subject='01', timestamp_tr=176, adj=[414, 414]), GraphEmo(x=[414], edge_index=[2, 170982], edge_attr=[170982], y=34, movie='AfterTheRain', subject='01', timestamp_tr=269, adj=[414, 414]), GraphEmo(x=[414], edge_index=[2, 170982], edge_attr=[170982], y=34, movie='AfterTheRain', subject='01', timestamp_tr=326, adj=[414, 414]), GraphEmo(x=[414], edge_index=[2, 170982], edge_attr=[170982], y=34, movie='AfterTheRain', subject='01', timestamp_tr=74, adj=[414, 414])

In [23]:
num_epochs = 2
tot_samples = len(dataset)
n_iteration = np.ceil(tot_samples/batch_size) #n_iteration in each epoch (how many times eahc epoch i update the weights)

for epoch in range(num_epochs):
    
    for i, (graphs, labels) in enumerate(dataloader): #each time it return a new batch

        if (i+1)%1 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{n_iteration}, Batch Size {batch_size}")
        #forward
        #backwards
        #update weigths

Epoch 1/2, Batch 1/10.0, Batch Size 32
Epoch 1/2, Batch 2/10.0, Batch Size 32
Epoch 1/2, Batch 3/10.0, Batch Size 32
Epoch 1/2, Batch 4/10.0, Batch Size 32
Epoch 1/2, Batch 5/10.0, Batch Size 32
Epoch 1/2, Batch 6/10.0, Batch Size 32
Epoch 1/2, Batch 7/10.0, Batch Size 32
Epoch 1/2, Batch 8/10.0, Batch Size 32
Epoch 1/2, Batch 9/10.0, Batch Size 32
Epoch 1/2, Batch 10/10.0, Batch Size 32
Epoch 2/2, Batch 1/10.0, Batch Size 32
Epoch 2/2, Batch 2/10.0, Batch Size 32
Epoch 2/2, Batch 3/10.0, Batch Size 32
Epoch 2/2, Batch 4/10.0, Batch Size 32
Epoch 2/2, Batch 5/10.0, Batch Size 32
Epoch 2/2, Batch 6/10.0, Batch Size 32
Epoch 2/2, Batch 7/10.0, Batch Size 32
Epoch 2/2, Batch 8/10.0, Batch Size 32
Epoch 2/2, Batch 9/10.0, Batch Size 32
Epoch 2/2, Batch 10/10.0, Batch Size 32
